## Exercise 02. Join

1. Create a connection to the database using the `sqlite3` library.
2. Create a new table called `datamart` in the database by joining the tables `pageviews` and `checker` using only one query.
   - The table should have the following columns: "uid", "labname", "first_commit_ts", and "first_view_ts".
   - "first_commit_ts" is a new name for the "timestamp" column in the checker table. It shows the first commit from a particular lab and user.
   - "first_view_ts" shows the first time a user visited the pageviews table. It is the timestamp of when a user visited the newsfeed.
   - `status = 'ready'` should still be a filter.
   - `numTrials = 1` should still be a filter.
   - "labnames" should be from the list: `laba04`, `laba04s`, `laba05`, `laba06`, `laba06s`, and `project1`.
   - The table should contain only users (uids with user_*), not admins.
   - "first_commit_ts" and "first_view_ts" should be parsed as `datetime64[ns]`.

In [ ]:
import pandas as pd
import sqlite3

con=sqlite3.connect('data/checking-logs.sqlite')
cursor=con.cursor()
create_table_query="""
CREATE TABLE IF NOT EXISTS datamart AS 
    WITH checker_temp AS(
        SELECT uid, labname, MIN(DATETIME(timestamp)) AS first_commit_ts
        FROM checker 
        WHERE status ='ready' AND numTrials=1 AND labname IN ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s', 'project1') AND
        uid LIKE "user_%"
        GROUP BY uid, labname
        ),
    pageviews_temp AS(
        SELECT uid, MIN(DATETIME(datetime)) AS first_view_ts
        FROM pageviews
        WHERE uid LIKE "user_%"
        GROUP BY uid
        )
    SELECT
    checker_temp.uid,
    checker_temp.labname,
    checker_temp.first_commit_ts,
    pageviews_temp.first_view_ts
FROM
    checker_temp
LEFT JOIN 
    pageviews_temp ON checker_temp.uid = pageviews_temp.uid
"""
cursor.execute("""DROP TABLE IF EXISTS datamart""")
cursor.execute(create_table_query)
datamart=pd.read_sql("""SELECT * FROM datamart""", con, parse_dates=['first_commit_ts', 'first_view_ts'])
datamart.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140 entries, 0 to 139
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              140 non-null    object        
 1   labname          140 non-null    object        
 2   first_commit_ts  140 non-null    datetime64[ns]
 3   first_view_ts    59 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 4.5+ KB


3. Using Pandas methods, create two dataframes: `test` and `control`.
   - `test` should contain users with values in "first_view_ts".
   - `control` should contain users with missing values in "first_view_ts".
   - Replace the missing values in the control table with the average "first_view_ts" value of the test users. We will use this value for future analyses.
   - Save both tables in the database. You will use them in the next exercises.

In [85]:
test=datamart[datamart['first_view_ts'].notna()].copy()
test
#test.info()


,uid,labname,first_commit_ts,first_view_ts
0,user_1,laba04,2020-04-26 17:06:18,2020-04-26 21:53:59
1,user_1,laba04s,2020-04-26 17:12:11,2020-04-26 21:53:59
2,user_1,laba05,2020-05-02 19:15:18,2020-04-26 21:53:59
3,user_1,laba06,2020-05-17 16:26:35,2020-04-26 21:53:59
4,user_1,laba06s,2020-05-20 12:23:37,2020-04-26 21:53:59
5,user_1,project1,2020-05-14 20:56:08,2020-04-26 21:53:59
6,user_10,laba04,2020-04-25 08:24:52,2020-04-18 12:19:50
7,user_10,laba04s,2020-04-25 08:37:54,2020-04-18 12:19:50
8,user_10,laba05,2020-05-01 19:27:26,2020-04-18 12:19:50
9,user_10,laba06,2020-05-19 11:39:28,2020-04-18 12:19:50


In [84]:
control=datamart[datamart['first_view_ts'].isna()].copy()
control.info()

<class 'pandas.core.frame.DataFrame'>
Index: 81 entries, 12 to 139
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              81 non-null     object        
 1   labname          81 non-null     object        
 2   first_commit_ts  81 non-null     datetime64[ns]
 3   first_view_ts    0 non-null      datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 3.2+ KB


In [79]:
control['first_view_ts'].fillna(test['first_view_ts'].mean(), inplace=True )
control

/tmp/ipykernel_361252/2289562314.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  control['first_view_ts'].fillna(test['first_view_ts'].mean(), inplace=True )


,uid,labname,first_commit_ts,first_view_ts
12,user_11,laba05,2020-05-03 21:06:55,2020-04-27 00:40:05.322033664
13,user_11,project1,2020-05-03 23:45:33,2020-04-27 00:40:05.322033664
14,user_12,laba04,2020-04-18 17:07:51,2020-04-27 00:40:05.322033664
15,user_12,laba04s,2020-04-26 15:42:38,2020-04-27 00:40:05.322033664
16,user_12,laba05,2020-05-03 08:39:25,2020-04-27 00:40:05.322033664
...,...,...,...,...
135,user_8,laba04s,2020-04-19 10:22:35,2020-04-27 00:40:05.322033664
136,user_8,laba05,2020-05-02 13:28:07,2020-04-27 00:40:05.322033664
137,user_8,laba06,2020-05-16 17:56:15,2020-04-27 00:40:05.322033664
138,user_8,laba06s,2020-05-16 20:01:07,2020-04-27 00:40:05.322033664


In [80]:
test.to_sql('test',con, if_exists='replace',index=True)
control.to_sql('control',con, if_exists='replace',index=True)

81

In [81]:
con.commit()
con.close()